# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [16]:
data[:10]

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


In [ ]:
## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
clean = data.groupby(by=['CustomerID', 'ProductName'], as_index=0)['Quantity'].sum()
clean

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
p_table = data.pivot_table(index='ProductID', columns = 'CustomerID', values='Quantity', fill_value = 0)
p_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,25,0,0,25
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,1,0,1,...,25,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,0,0,0,0,0,0,0,0,0,0,...,0,0,25,0,25,0,25,0,0,0
450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,25,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
customer_s_matrix = pd.DataFrame(squareform(pdist(p_table, metric='euclidean')))
customer_s_matrix

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,230.497288,244.020491,239.841614,235.295559,228.512582,242.274638,228.551963,217.731486,240.981327,...,229.117874,221.054292,244.374712,234.731762,236.721355,224.937769,222.441003,234.247732,237.825566,230.421353
1,230.497288,0.000000,222.968608,234.961699,226.077420,226.532558,230.651252,222.816068,204.606940,225.559305,...,221.697993,213.733479,242.008264,224.138350,236.304888,220.349722,229.327277,223.541942,229.024016,219.888608
2,244.020491,222.968608,0.000000,231.451939,221.485891,223.705163,234.407764,220.775904,211.936311,222.441003,...,230.436542,228.309001,238.203694,245.097940,234.049140,220.814402,232.219724,233.289520,225.816297,224.788790
3,239.841614,234.961699,231.451939,0.000000,233.353809,239.553752,248.706655,229.115691,220.356529,225.588120,...,229.165879,233.540146,234.646543,251.525347,232.686484,224.866627,242.507732,236.512156,228.912647,237.911748
4,235.295559,226.077420,221.485891,233.353809,0.000000,219.152915,237.025315,227.107904,213.468967,227.596134,...,224.497216,223.515100,237.029534,225.590337,230.579704,217.345347,235.669260,221.106309,232.978540,230.060862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,224.937769,220.349722,220.814402,224.866627,217.345347,214.287190,227.754254,230.922065,213.700725,224.470488,...,231.006493,212.179170,227.560981,238.759293,230.716276,0.000000,225.483924,216.704868,224.735400,224.448212
448,222.441003,229.327277,232.219724,242.507732,235.669260,229.185514,241.902873,234.034186,218.611528,226.609797,...,231.354706,226.589938,233.291663,233.540146,240.696905,225.483924,0.000000,227.763035,232.733753,230.225976
449,234.247732,223.541942,233.289520,236.512156,221.106309,223.714103,234.923392,227.890324,206.961349,231.685131,...,219.310282,222.335332,233.719918,232.510215,226.404505,216.704868,227.763035,0.000000,229.667150,227.253163
450,237.825566,229.024016,225.816297,228.912647,232.978540,226.501656,230.573199,229.784682,213.171293,236.750924,...,223.624686,218.160491,233.422364,234.963827,238.344289,224.735400,232.733753,229.667150,0.000000,226.439837


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
dist_calculation = pdist(X=p_table, metric='euclidean')
dist_calculation

array([230.49728849, 244.02049094, 239.8416144 , ..., 229.66715046,
       227.25316279, 226.43983748])

In [11]:
dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        , 230.49728849, 244.02049094, ..., 234.24773211,
        237.82556633, 230.42135318],
       [230.49728849,   0.        , 222.96860766, ..., 223.54194237,
        229.02401621, 219.88860816],
       [244.02049094, 222.96860766,   0.        , ..., 233.2895197 ,
        225.81629702, 224.78878976],
       ...,
       [234.24773211, 223.54194237, 233.2895197 , ...,   0.        ,
        229.66715046, 227.25316279],
       [237.82556633, 229.02401621, 225.81629702, ..., 229.66715046,
          0.        , 226.43983748],
       [230.42135318, 219.88860816, 224.78878976, ..., 227.25316279,
        226.43983748,   0.        ]])

In [14]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=customer_s_matrix.columns, 
                           columns=customer_s_matrix.columns)
euclid_dist

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,230.497288,244.020491,239.841614,235.295559,228.512582,242.274638,228.551963,217.731486,240.981327,...,229.117874,221.054292,244.374712,234.731762,236.721355,224.937769,222.441003,234.247732,237.825566,230.421353
1,230.497288,0.000000,222.968608,234.961699,226.077420,226.532558,230.651252,222.816068,204.606940,225.559305,...,221.697993,213.733479,242.008264,224.138350,236.304888,220.349722,229.327277,223.541942,229.024016,219.888608
2,244.020491,222.968608,0.000000,231.451939,221.485891,223.705163,234.407764,220.775904,211.936311,222.441003,...,230.436542,228.309001,238.203694,245.097940,234.049140,220.814402,232.219724,233.289520,225.816297,224.788790
3,239.841614,234.961699,231.451939,0.000000,233.353809,239.553752,248.706655,229.115691,220.356529,225.588120,...,229.165879,233.540146,234.646543,251.525347,232.686484,224.866627,242.507732,236.512156,228.912647,237.911748
4,235.295559,226.077420,221.485891,233.353809,0.000000,219.152915,237.025315,227.107904,213.468967,227.596134,...,224.497216,223.515100,237.029534,225.590337,230.579704,217.345347,235.669260,221.106309,232.978540,230.060862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,224.937769,220.349722,220.814402,224.866627,217.345347,214.287190,227.754254,230.922065,213.700725,224.470488,...,231.006493,212.179170,227.560981,238.759293,230.716276,0.000000,225.483924,216.704868,224.735400,224.448212
448,222.441003,229.327277,232.219724,242.507732,235.669260,229.185514,241.902873,234.034186,218.611528,226.609797,...,231.354706,226.589938,233.291663,233.540146,240.696905,225.483924,0.000000,227.763035,232.733753,230.225976
449,234.247732,223.541942,233.289520,236.512156,221.106309,223.714103,234.923392,227.890324,206.961349,231.685131,...,219.310282,222.335332,233.719918,232.510215,226.404505,216.704868,227.763035,0.000000,229.667150,227.253163
450,237.825566,229.024016,225.816297,228.912647,232.978540,226.501656,230.573199,229.784682,213.171293,236.750924,...,223.624686,218.160491,233.422364,234.963827,238.344289,224.735400,232.733753,229.667150,0.000000,226.439837


In [20]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(data, 'euclidean'))),
                                index=customer_s_matrixs.columns,
                                columns=customer_s_matrix.columns)

euclid_dist_norm

ValueError: Unsupported dtype object

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.